In [1]:
import metquest
import networkx as nx
from collections import Counter
import os
import pandas as pd 
import pickle


In [2]:
def find_metabolic_support_index(path, path_single, graph_name_combined, graph_name_firstorg, graph_name_secondorg, seed_metabolites):
    combined_graph = nx.read_gpickle(path + graph_name_combined) #seed_mets.txt'
    first_org_graph = nx.read_gpickle(path_single + graph_name_firstorg)
    second_org_graph = nx.read_gpickle(path_single + graph_name_secondorg)

    lb_m_org_1, status_dict_org_1, scope_org_1 = metquest.forward_pass(first_org_graph, seed_metabolites)
    org_1_reactions = []
    org_1_metabolites = []
    node_attr = nx.get_node_attributes(first_org_graph, 'bipartite')
    for nodes in first_org_graph:
        if node_attr[nodes] == 1:
            org_1_reactions.append(nodes)
        else:
            metname = nodes.split(' ')[0]
            if metname not in org_1_metabolites and '[e]' not in metname:
                org_1_metabolites.append(metname)

    
    lb_m_org_2, status_dict_org_2, scope_org_2 = metquest.forward_pass(second_org_graph, seed_metabolites)
    org_2_reactions = []
    org_2_metabolites = []
    node_attr = nx.get_node_attributes(second_org_graph, 'bipartite')
    for nodes in second_org_graph:
        if node_attr[nodes] == 1:
            org_2_reactions.append(nodes)
        else:
            metname = nodes.split(' ')[0]
            if metname not in org_2_metabolites and '[e]' not in metname:
                org_2_metabolites.append(metname)
    
    org_1 = graph_name_firstorg.split('_.gpickle')[0]
    org_2 = graph_name_secondorg.split('_.gpickle')[0]
    reactions = []
    lb_m, status_dict, scope = metquest.forward_pass(combined_graph, seed_metabolites)
    reactions = []
    node_attr = nx.get_node_attributes(combined_graph, 'bipartite')
    for nodes in combined_graph:
        if node_attr[nodes] == 1:
            reactions.append(nodes)

    stuck_internal_rxns_in_org_1_combined = []
    stuck_internal_rxns_in_org_2_combined = []
    stuck_external_rxns_in_org_1_combined = set([])
    stuck_external_rxns_in_org_2_combined = set([])
    # Combined stuck nodes
    all_stuck_nodes = set(reactions) - set(status_dict)
    for nodes in all_stuck_nodes:
        nodesplit = nodes.split(' ')[1]
        if org_2 in nodes:
            if 'IR' in nodesplit or 'RR' in nodesplit or 'RevBR' in nodesplit:
                if 'ER' not in nodesplit:
                    stuck_internal_rxns_in_org_2_combined.append(nodes)
                else:
                    stuck_external_rxns_in_org_2_combined.add(nodes)
            else:
                 stuck_external_rxns_in_org_2_combined.add(nodes)
   
        elif org_1 in nodes:
            if 'IR' in nodesplit or 'RR' in nodesplit or 'RevBR' in nodesplit:
                if 'ER' not in nodesplit:
                    stuck_internal_rxns_in_org_1_combined.append(nodes)
                else:
                    stuck_external_rxns_in_org_1_combined.add(nodes)
            else:
                stuck_external_rxns_in_org_1_combined.add(nodes)


    org_2_stuck_nodes = set(org_2_reactions) - set(status_dict_org_2)
    org_2_stuck_internal_single = []
    org_2_stuck_external = []
    for nodes in org_2_stuck_nodes:
        nodesplit = nodes.split(' ')[1]
        if 'IR' in nodes or 'RR' in nodes or 'RevBR' in nodes:
            if 'ER' not in nodes:
                org_2_stuck_internal_single.append(nodes)

            else: #ERR nodes
                org_2_stuck_external.append(nodes)
        else:
            org_2_stuck_external.append(nodes)
    org_1_stuck_nodes_internal = set(org_1_reactions) - set(status_dict_org_1) #Status dict has all visited nodes
    org_1_stuck_internal_single = []
    org_1_stuck_external = []
    for nodes in org_1_stuck_nodes_internal:
        nodesplit = nodes.split(' ')[1]
        if 'IR' in nodesplit or 'RR' in nodesplit or 'RevBR' in nodesplit: 
            if 'ER' not in nodesplit: #To avoid entries like NCERR and ERR
                org_1_stuck_internal_single.append(nodesplit)
            else:
                org_1_stuck_external.append(nodesplit)
        else:
            org_1_stuck_external.append(nodesplit)
    
    try:
        msi_org1 = round(1-(len(stuck_internal_rxns_in_org_1_combined)/ len(org_1_stuck_internal_single)), 6)
        msi_org2 = round(1-(len(stuck_internal_rxns_in_org_2_combined)/ len(org_2_stuck_internal_single)), 6)

        print('stuck_external_rxns_in_org_1_combined ' + org_1, len(stuck_external_rxns_in_org_1_combined))
        print('stuck_external_rxns_in_org_2_combined ' + org_2, len(stuck_external_rxns_in_org_2_combined))
        print('stuck_internal_rxns_in_org_2_combined ' + org_2, len(stuck_internal_rxns_in_org_2_combined))
        print('stuck_internal_rxns_in_org_1_combined ' + org_1, len(stuck_internal_rxns_in_org_1_combined))
        print('org_1_stuck_internal_single ' + org_1, len(org_1_stuck_internal_single))
        print('org_2_stuck_internal_single ' + org_2, len(org_2_stuck_internal_single))
        print('org_2_stuck_external ' + org_2, len(org_2_stuck_external))
        print('org_1_stuck_external ' + org_1, len(org_1_stuck_external))
        print('Metabolic support index ' + org_1, msi_org1) 
        print('Metabolic support index ' + org_2, msi_org2)
        if len(splitfilenames) == 1:
            df.at[splitfilenames[0], splitfilenames[0]] = 0
        elif len(splitfilenames) >= 1:
            df.at[splitfilenames[0], splitfilenames[1]] = msi_org1 
            df.at[splitfilenames[1], splitfilenames[0]] = msi_org2
    except:
        print('Zero Error')


In [3]:
current_working_directory = os.getcwd()
path = '/Users/ashnanarain/Computational_Biology/Gut_Microbiome/Networks/Models_Double/'
path_single = '/Users/ashnanarain/Computational_Biology/Gut_Microbiome/Networks/Models_Single/'
seed_metabolite_fname = '/Users/ashnanarain/Computational_Biology/Gut_Microbiome/Diets_and_Media/Medium_DMEM_mets.csv'
org_names_fname =  '/Users/ashnanarain/Computational_Biology/Gut_Microbiome/Models.csv'
res_paths = os.path.join(current_working_directory, 'MSI_Results')

In [4]:
if not os.path.exists(res_paths):
    os.makedirs(res_paths)
with open(org_names_fname, 'r') as f:
    org_names = f.read().splitlines()
org_names_without_xml = []
for names in org_names:
    org_names_without_xml.append(names.strip('.xml'))
row_names = pd.Index(org_names_without_xml, name='rows')
column_names = pd.Index(org_names_without_xml, name='columns')
df = pd.DataFrame(index=row_names, columns=column_names) #df with all the scope entries

with open(seed_metabolite_fname, 'r') as f:
    seed_mets_from_file = f.read().splitlines()

In [6]:
for graphname in os.listdir(path):
    if graphname.endswith('.gpickle'):
        splitfilenames = []
        for names in org_names:
            if names.split('.xml')[0] in graphname:
                splitfilenames.append(names.split('.xml')[0])
        
        graph_name_firstorg = splitfilenames[0] + '_.gpickle'
        graph_name_secondorg = splitfilenames[1] + '_.gpickle'

        '''
        file_1 = '/Users/ashnanarain/Computational_Biology/Gut_Microbiome/Models/' + splitfilenames[0] + '.xml'
        file_2 = '/Users/ashnanarain/Computational_Biology/Gut_Microbiome/Models/' + splitfilenames[1] + '.xml'  

        model_1 = read_sbml_model(file_1)
        model_2 = read_sbml_model(file_2)

        table = {'id': ['model_1', 'model_2'], 'file':[file_1, file_2]}

        data = pd.DataFrame(table)      
        taxonomy = data 
        com = Community(taxonomy, progress = False)

        medium = minimal_medium(com, 1)
        '''
        '''
        medium_mets = []
        for i in medium.index:
            medium_mets.append(i.replace('_m', '[e]'))
        medium_mets

        seed_mets = set(medium_mets)
        for fname in splitfilenames:
            for mets in medium_mets:
                seed_mets.add(fname + ' ' + mets)
          
        ''' 
        seed_mets = set(seed_mets_from_file)
        for fname in splitfilenames:
            for mets in seed_mets_from_file:
                seed_mets.add(fname + ' ' + mets)
        
        find_metabolic_support_index(path, path_single, graphname, graph_name_firstorg, graph_name_secondorg, seed_mets)

#df.to_csv('MSI_DMEM.csv')


stuck_external_rxns_in_org_1_combined Alistipes_finegoldii_DSM_17242 202
stuck_external_rxns_in_org_2_combined Megasphaera_elsdenii_DSM_20460 198
stuck_internal_rxns_in_org_2_combined Megasphaera_elsdenii_DSM_20460 1482
stuck_internal_rxns_in_org_1_combined Alistipes_finegoldii_DSM_17242 1450
org_1_stuck_internal_single Alistipes_finegoldii_DSM_17242 1450
org_2_stuck_internal_single Megasphaera_elsdenii_DSM_20460 1482
org_2_stuck_external Megasphaera_elsdenii_DSM_20460 198
org_1_stuck_external Alistipes_finegoldii_DSM_17242 202
Metabolic support index Alistipes_finegoldii_DSM_17242 0.0
Metabolic support index Megasphaera_elsdenii_DSM_20460 0.0
stuck_external_rxns_in_org_1_combined Bacteroides_fragilis_NCTC_9343 932
stuck_external_rxns_in_org_2_combined Dialister_invisus_DSM_15470 140
stuck_internal_rxns_in_org_2_combined Dialister_invisus_DSM_15470 1125
stuck_internal_rxns_in_org_1_combined Bacteroides_fragilis_NCTC_9343 2573
org_1_stuck_internal_single Bacteroides_fragilis_NCTC_9343 2

stuck_external_rxns_in_org_1_combined Clostridium_clostridioforme_CM201 890
stuck_external_rxns_in_org_2_combined Coprococcus_catus_GD_7 190
stuck_internal_rxns_in_org_2_combined Coprococcus_catus_GD_7 1381
stuck_internal_rxns_in_org_1_combined Clostridium_clostridioforme_CM201 1936
org_1_stuck_internal_single Clostridium_clostridioforme_CM201 1936
org_2_stuck_internal_single Coprococcus_catus_GD_7 1385
org_2_stuck_external Coprococcus_catus_GD_7 194
org_1_stuck_external Clostridium_clostridioforme_CM201 890
Metabolic support index Clostridium_clostridioforme_CM201 0.0
Metabolic support index Coprococcus_catus_GD_7 0.002888
stuck_external_rxns_in_org_1_combined Bacteroides_intestinalis_341_DSM_17393 804
stuck_external_rxns_in_org_2_combined Lactobacillus_mucosae_LM1 162
stuck_internal_rxns_in_org_2_combined Lactobacillus_mucosae_LM1 1145
stuck_internal_rxns_in_org_1_combined Bacteroides_intestinalis_341_DSM_17393 2791
org_1_stuck_internal_single Bacteroides_intestinalis_341_DSM_17393 2

stuck_external_rxns_in_org_1_combined Alistipes_putredinis_DSM_17216 120
stuck_external_rxns_in_org_2_combined Bacteroides_uniformis_ATCC_8492 864
stuck_internal_rxns_in_org_2_combined Bacteroides_uniformis_ATCC_8492 2893
stuck_internal_rxns_in_org_1_combined Alistipes_putredinis_DSM_17216 1030
org_1_stuck_internal_single Alistipes_putredinis_DSM_17216 1034
org_2_stuck_internal_single Bacteroides_uniformis_ATCC_8492 2897
org_2_stuck_external Bacteroides_uniformis_ATCC_8492 866
org_1_stuck_external Alistipes_putredinis_DSM_17216 122
Metabolic support index Alistipes_putredinis_DSM_17216 0.003868
Metabolic support index Bacteroides_uniformis_ATCC_8492 0.001381
stuck_external_rxns_in_org_1_combined Bacteroides_fragilis_NCTC_9343 922
stuck_external_rxns_in_org_2_combined Enterobacter_cloacae_EcWSU1 332
stuck_internal_rxns_in_org_2_combined Enterobacter_cloacae_EcWSU1 2129
stuck_internal_rxns_in_org_1_combined Bacteroides_fragilis_NCTC_9343 2543
org_1_stuck_internal_single Bacteroides_fragi

stuck_external_rxns_in_org_1_combined Haemophilus_parainfluenzae_T3T1 188
stuck_external_rxns_in_org_2_combined Roseburia_intestinalis_L1_82 892
stuck_internal_rxns_in_org_2_combined Roseburia_intestinalis_L1_82 1809
stuck_internal_rxns_in_org_1_combined Haemophilus_parainfluenzae_T3T1 1413
org_1_stuck_internal_single Haemophilus_parainfluenzae_T3T1 1413
org_2_stuck_internal_single Roseburia_intestinalis_L1_82 1813
org_2_stuck_external Roseburia_intestinalis_L1_82 896
org_1_stuck_external Haemophilus_parainfluenzae_T3T1 188
Metabolic support index Haemophilus_parainfluenzae_T3T1 0.0
Metabolic support index Roseburia_intestinalis_L1_82 0.002206
stuck_external_rxns_in_org_1_combined Blautia_wexlerae_DSM_19850 194
stuck_external_rxns_in_org_2_combined Prevotella_ruminicola_23 822
stuck_internal_rxns_in_org_2_combined Prevotella_ruminicola_23 2717
stuck_internal_rxns_in_org_1_combined Blautia_wexlerae_DSM_19850 1377
org_1_stuck_internal_single Blautia_wexlerae_DSM_19850 1377
org_2_stuck_in

stuck_external_rxns_in_org_1_combined Bifidobacterium_adolescentis_ATCC_15703 254
stuck_external_rxns_in_org_2_combined Parabacteroides_johnsonii_DSM_18315 806
stuck_internal_rxns_in_org_2_combined Parabacteroides_johnsonii_DSM_18315 2860
stuck_internal_rxns_in_org_1_combined Bifidobacterium_adolescentis_ATCC_15703 1121
org_1_stuck_internal_single Bifidobacterium_adolescentis_ATCC_15703 1121
org_2_stuck_internal_single Parabacteroides_johnsonii_DSM_18315 2872
org_2_stuck_external Parabacteroides_johnsonii_DSM_18315 812
org_1_stuck_external Bifidobacterium_adolescentis_ATCC_15703 254
Metabolic support index Bifidobacterium_adolescentis_ATCC_15703 0.0
Metabolic support index Parabacteroides_johnsonii_DSM_18315 0.004178
stuck_external_rxns_in_org_1_combined Bacteroides_thetaiotaomicron_VPI_5482 1014
stuck_external_rxns_in_org_2_combined Faecalibacterium_prausnitzii_L2_6 820
stuck_internal_rxns_in_org_2_combined Faecalibacterium_prausnitzii_L2_6 1614
stuck_internal_rxns_in_org_1_combined B

...

stuck_external_rxns_in_org_1_combined Coprococcus_catus_GD_7 184
stuck_external_rxns_in_org_2_combined Klebsiella_pneumoniae_pneumoniae_MGH78578 362
stuck_internal_rxns_in_org_2_combined Klebsiella_pneumoniae_pneumoniae_MGH78578 2159
stuck_internal_rxns_in_org_1_combined Coprococcus_catus_GD_7 1376
org_1_stuck_internal_single Coprococcus_catus_GD_7 1385
org_2_stuck_internal_single Klebsiella_pneumoniae_pneumoniae_MGH78578 2159
org_2_stuck_external Klebsiella_pneumoniae_pneumoniae_MGH78578 362
org_1_stuck_external Coprococcus_catus_GD_7 194
Metabolic support index Coprococcus_catus_GD_7 0.006498
Metabolic support index Klebsiella_pneumoniae_pneumoniae_MGH78578 0.0
stuck_external_rxns_in_org_1_combined Bifidobacterium_longum_NCC2705 228
stuck_external_rxns_in_org_2_combined Megasphaera_elsdenii_DSM_20460 198
stuck_internal_rxns_in_org_2_combined Megasphaera_elsdenii_DSM_20460 1482
stuck_internal_rxns_in_org_1_combined Bifidobacterium_longum_NCC2705 1033
org_1_stuck_internal_single Bifido

stuck_external_rxns_in_org_1_combined Escherichia_coli_str_K_12_substr_MG1655 1036
stuck_external_rxns_in_org_2_combined Ruminococcus_torques_ATCC_27756 170
stuck_internal_rxns_in_org_2_combined Ruminococcus_torques_ATCC_27756 1097
stuck_internal_rxns_in_org_1_combined Escherichia_coli_str_K_12_substr_MG1655 3345
org_1_stuck_internal_single Escherichia_coli_str_K_12_substr_MG1655 3345
org_2_stuck_internal_single Ruminococcus_torques_ATCC_27756 1105
org_2_stuck_external Ruminococcus_torques_ATCC_27756 178
org_1_stuck_external Escherichia_coli_str_K_12_substr_MG1655 1036
Metabolic support index Escherichia_coli_str_K_12_substr_MG1655 0.0
Metabolic support index Ruminococcus_torques_ATCC_27756 0.00724
stuck_external_rxns_in_org_1_combined Alistipes_finegoldii_DSM_17242 196
stuck_external_rxns_in_org_2_combined Clostridium_clostridioforme_CM201 890
stuck_internal_rxns_in_org_2_combined Clostridium_clostridioforme_CM201 1936
stuck_internal_rxns_in_org_1_combined Alistipes_finegoldii_DSM_172

stuck_external_rxns_in_org_1_combined Akkermansia_muciniphila_ATCC_BAA_835 212
stuck_external_rxns_in_org_2_combined Clostridium_clostridioforme_CM201 890
stuck_internal_rxns_in_org_2_combined Clostridium_clostridioforme_CM201 1936
stuck_internal_rxns_in_org_1_combined Akkermansia_muciniphila_ATCC_BAA_835 1399
org_1_stuck_internal_single Akkermansia_muciniphila_ATCC_BAA_835 1403
org_2_stuck_internal_single Clostridium_clostridioforme_CM201 1936
org_2_stuck_external Clostridium_clostridioforme_CM201 890
org_1_stuck_external Akkermansia_muciniphila_ATCC_BAA_835 214
Metabolic support index Akkermansia_muciniphila_ATCC_BAA_835 0.002851
Metabolic support index Clostridium_clostridioforme_CM201 0.0
stuck_external_rxns_in_org_1_combined Alistipes_shahii_WAL_8301 774
stuck_external_rxns_in_org_2_combined Phascolarctobacterium_succinatutens_YIT_12067 150
stuck_internal_rxns_in_org_2_combined Phascolarctobacterium_succinatutens_YIT_12067 1248
stuck_internal_rxns_in_org_1_combined Alistipes_shahi

stuck_external_rxns_in_org_1_combined Shigella_flexneri_2002017 976
stuck_external_rxns_in_org_2_combined Veillonella_atypica_ACS_049_V_Sch6 194
stuck_internal_rxns_in_org_2_combined Veillonella_atypica_ACS_049_V_Sch6 1380
stuck_internal_rxns_in_org_1_combined Shigella_flexneri_2002017 3239
org_1_stuck_internal_single Shigella_flexneri_2002017 3239
org_2_stuck_internal_single Veillonella_atypica_ACS_049_V_Sch6 1388
org_2_stuck_external Veillonella_atypica_ACS_049_V_Sch6 198
org_1_stuck_external Shigella_flexneri_2002017 976
Metabolic support index Shigella_flexneri_2002017 0.0
Metabolic support index Veillonella_atypica_ACS_049_V_Sch6 0.005764
stuck_external_rxns_in_org_1_combined Bacteroides_fragilis_NCTC_9343 918
stuck_external_rxns_in_org_2_combined Escherichia_coli_str_K_12_substr_MG1655 1034
stuck_internal_rxns_in_org_2_combined Escherichia_coli_str_K_12_substr_MG1655 3341
stuck_internal_rxns_in_org_1_combined Bacteroides_fragilis_NCTC_9343 2501
org_1_stuck_internal_single Bactero

KeyboardInterrupt: 

In [ ]:
medium_mets = []
for i in medium.index:
    medium_mets.append(i.replace('_m', '[e]'))
medium_mets